In [1]:
import sys
import os
import stim, numpy as np
sys.path.append('/Users/a46668993/Desktop/corrqec')
from src.noisemodel import LongTimePairPoly, LongTimeStreakPoly, LongTimePairMPoly, LongTimeStreakMPoly, LongTimePairCPoly, LongTimePairCPoly, LongTimeStreakCPoly

In [13]:
def make_surface_code_memory_circuit(distance, rounds, rotated=True):
    code = "surface_code:rotated_memory_z" if rotated else "surface_code:unrotated_memory_z"
    circuit = stim.Circuit.generated(
        code,
        rounds=rounds,
        distance=distance,
        # No SCL noise kwargs here: we add noise ourselves via FlipSimulator
    )
    return circuit

circuit = make_surface_code_memory_circuit(distance=3, rounds=3, rotated=True)
#print(circuit.to_str()[:500])  # sanity check: first few lines
#print(circuit)
circuit.diagram()

/-----------------\     /---\ /---\ /-----\   /---------------------------------\ /REP 2       /---\ /---\ /-----\   /------------------------------------------------------------------------------\ \ /--------------------------------------------------------------------------------------------------------\
 q0: -----------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------
                                                                                        |                                                                                                                   |
 q1: -QUBIT_COORDS(1,1)-R-----X---------------@-----------------------------------------|------------X---------------@--------------------------------------------------------------------------------------|-M:rec[24]---------------------------------------------------OBSERVABLE_INCLUDE:L0*=rec[26]*rec[25]*rec[24]-
                              |               |                                         |            |               |                                                                                      |
 q2: -QUBIT_COORDS(2,0)-R-H-@-@---------------|---H-MR:rec[0]---------------------------|--------H-@-@---------------|---H-MR:rec[8+iter*8]--DETECTOR(2,0,1+iter):D[4+iter*8]=rec[8+iter*8]*rec[0+iter*8]---|------------------------------------------------------------------------------------------------------------
                            |                 |                                         |          |                 |                                                                                      |
 q3: -QUBIT_COORDS(3,1)-R---X-----@---------X-|-----------------------------------------|----------X-----@---------X-|--------------------------------------------------------------------------------------|-M:rec[25]--------------------------------------------------------------------------------------------------
                                  |         | |                                         |                |         | |                                                                                      |
 q4: -----------------------------|---------|-|-----------------------------------------|----------------|---------|-|--------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------
                                  |         | |                                         |                |         | |                                                                                      |
 q5: -QUBIT_COORDS(5,1)-R---------|---X-----|-|-@---------------------------------------|----------------|---X-----|-|-@------------------------------------------------------------------------------------|-M:rec[26]--------------------------------------------------------------------------------------------------
                                  |   |     | | |                                       |                |   |     | | |                                                                                    |
 q6: -----------------------------|---|-----|-|-|---------------------------------------|----------------|---|-----|-|-|------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------
                                  |   |     | | |                                       |                |   |     | | |                                                                                    |
 q7: -----------------------------|---|-----|-|-|----

In [14]:
# Example parameters: tweak these later to match the paper
A = 0.5     # overall scale
p = 0.01    # base error scale
n = 2       # for poly: exponent; for exp: base in denominator

# Noise on data qubits only, depolarizing errors
noise_model = LongTimePairPoly(
    A=A,
    p=p,
    n=n,
    noisy_qubits="all",          # or "all" / "syndrome"
    error_type="depolarizing",    # or "X", "Z", etc.
)

shots = 100000  # number of independent trajectories (batch size)

detection_events, observable_flips = noise_model.sample_circuit(
    circuit=circuit,
    shots=shots,
)

print("detection_events shape:", detection_events.shape)  # (shots, num_detectors)
print("observable_flips shape:", observable_flips.shape)  # (shots,)

detection_events shape: (100000, 24)
observable_flips shape: (100000,)


In [16]:
noisy_circuit = noise_model.convert_circuit_marginalised(circuit)
noisy_circuit.diagram()

/-----------------\ /-----------------------\   /---\ /---\ /-----\   /---------------------------------------------------------\     /---\ /---\ /-----\   /-------------------------------------------------------------------\     /---\ /---\ /-----\   /--------------------------------------------------------------------------------------------------------\
 q0: ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

 q1: -QUBIT_COORDS(1,1)-R-DEPOLARIZE1(0.00499467)-----X---------------@---------------DEPOLARIZE1(0.00797867)-------------------------------X---------------@----------------DEPOLARIZE1(0.00499467)----------------------------------------X---------------@----------------M:rec[24]---------------------------OBSERVABLE_INCLUDE:L0*=rec[26]*rec[25]*rec[24]--------------
                                                      |               |                                                                     |               |                                                                               |               |
 q2: -QUBIT_COORDS(2,0)-R-DEPOLARIZE1(0.00499467)-H-@-@---------------|---H-MR:rec[0]-DEPOLARIZE1(0.00797867)---------------------------H-@-@---------------|---H-MR:rec[8]--DETECTOR(2,0,1):D4=rec[8]*rec[0]---DEPOLARIZE1(0.00499467)-H-@-@---------------|---H-MR:rec[16]-DETECTOR(2,0,2):D12=rec[16]*rec[8]--------------------------------------------------------------
                                                    |                 |                                                                   |                 |                                                                             |                 |
 q3: -QUBIT_COORDS(3,1)-R-DEPOLARIZE1(0.00499467)---X-----@---------X-|---------------DEPOLARIZE1(0.00797867)-----------------------------X-----@---------X-|----------------DEPOLARIZE1(0.00499467)--------------------------------------X-----@---------X-|----------------M:rec[25]---------------------------------------------------------------------------------------
                                                          |         | |                                                                         |         | |                                                                                   |         | |
 q4: -----------------------------------------------------|---------|-|-------------------------------------------------------------------------|---------|-|-----------------------------------------------------------------------------------|---------|-|----------------------------------------------------------------------------------------------------------------
                                                          |         | |                                                                         |         | |                                                                                   |         | |
 q5: -QUBIT_COORDS(5,1)-R-DEPOLARIZE1(0.00499467)---------|---X-----|-|-@-------------DEPOLARIZE1(0.00797867)-----------------------------------|---X-----|-|-@--------------DEPOLARIZE1(0.00499467)--------------------------------------------|---X-----|-|-@--------------M:rec[26]---------------------------------------------------------------------------------------
                                                          |   |     | | |                                                                       |   |     | | |                                                                                 |   |     | | |
 q6: -----------------------------------------------------|---|-----|-|-|-----------------------------------------------------------------------|---|-----|-|-|-------------

In [45]:
0.0049+0.0079+0.0049

0.0177

In [18]:
detection_events=detection_events.astype(np.uint8)

non_zero_indices = np.argwhere(detection_events > 0)
for shot, detector in non_zero_indices:
    print(f"Shot: {shot}, Detector: {detector}")


Shot: 10, Detector: 5
Shot: 14, Detector: 11
Shot: 14, Detector: 19
Shot: 28, Detector: 2
Shot: 28, Detector: 11
Shot: 33, Detector: 7
Shot: 35, Detector: 3
Shot: 35, Detector: 7
Shot: 37, Detector: 11
Shot: 39, Detector: 5
Shot: 39, Detector: 6
Shot: 39, Detector: 9
Shot: 39, Detector: 10
Shot: 39, Detector: 13
Shot: 39, Detector: 18
Shot: 49, Detector: 6
Shot: 49, Detector: 14
Shot: 74, Detector: 6
Shot: 74, Detector: 14
Shot: 80, Detector: 14
Shot: 83, Detector: 1
Shot: 83, Detector: 2
Shot: 83, Detector: 11
Shot: 83, Detector: 13
Shot: 94, Detector: 5
Shot: 94, Detector: 12
Shot: 106, Detector: 11
Shot: 108, Detector: 15
Shot: 120, Detector: 2
Shot: 125, Detector: 11
Shot: 125, Detector: 19
Shot: 131, Detector: 2
Shot: 131, Detector: 10
Shot: 132, Detector: 15
Shot: 134, Detector: 2
Shot: 134, Detector: 3
Shot: 134, Detector: 6
Shot: 140, Detector: 18
Shot: 143, Detector: 10
Shot: 143, Detector: 18
Shot: 147, Detector: 0
Shot: 147, Detector: 1
Shot: 147, Detector: 17
Shot: 148, Det

In [19]:
repetitions = 50     # how many batches to average over
batch_size = 1000    # shots per repetition

logical_error_rate = noise_model.sample_logical_error_rate(
    circuit=circuit,
    repetitions=repetitions,
    batch_size=batch_size,
    max_errors=None,           # or some integer to early-stop
    return_error_rate=True,    # or False to get (total_shots, total_errors)
    detector_error_model=None, # let it build a marginalised DEM
)

print("for physical error rate,", p, ",Estimated logical error rate:", logical_error_rate)

for physical error rate, 0.01 ,Estimated logical error rate: 0.0008799999999999999
